# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh
from DisasterConfig import DBSettings
from DisasterConfig import TokenizerSettings

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC

from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///{}'.format(DBSettings.Database))

df = pd.read_sql_table(DBSettings.Table, engine)
#df = pd.read_sql("SELECT * FROM {}".format(DBSettings.Table), engine)

In [4]:
dsh.AnalyzeDataFrame(df)

-------------------- Dataframe analysis started --------------------
Shape:  (26216, 39)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 38, 97.44%
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64                  0.0
money, type: int64                     0.0
missing_people, type: int64        

Hmm... there were nan values that have been inserted into database...

In [5]:
df[df.original == np.nan]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Hmmm.... there were definitely missing values...

In [6]:
type(df.at[7408, 'original']) # example ID I radonmly found.

str

In [7]:
df[df.original == None]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [8]:
type(None)

NoneType

Hmmm.... I thought that None or np.nan would be interpretet as something like DBNULL. Instead it is a string value?! I didn't calculated that...

In [9]:
df[df.original == 'NoneType']

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


I give up... I thought there is an adequate handling for np.nan und database NULL values. I just intended to check the number of mising values before and after writing into database. But as you can see, currently I'm out of ideas how to do it ... And I don't intend to waste time with that.

As I mentioned at ETL 7.0, I didn't drop rows that were irrelevant because its possible that irrelevant rows have been added before selecting the data from database. 

So here at this step the data needs to be cleaned either way. My following steps:
- remove column 'original' as there are many missing values as explored during ETL and cannot be proven here as you can see above ^^
- remove rows when message starts with 'NOTES:'
- remove rows with related not in [0, 1]
- remove rows with messages that are much longer than the length of all messages (I intend to predict messages, no prosa texts)

In [10]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
Columns found to remove:  ['original']
-------------------- Finished removing columns matchting to wildcards --------------------


In [11]:
df[df['message'].str.startswith('NOTES:')]

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3066,3539,NOTES: Personal. Not an emergency.,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3099,3579,NOTES: aclet dry We are counting on you.,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3103,3583,NOTES: talking about gas prices,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3166,3651,NOTES: Historic earthquake in Haiti.,direct,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3240,3737,NOTES: They are requesting that somebody to pa...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3262,3761,NOTES: Regular gossip or message sharing. Not ...,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3272,3775,NOTES: personal request. Not an emergency.,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3315,3820,NOTES: regular gossip. Not an emergency.,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3361,3871,NOTES: Gossip. Not an emergency.,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3366,3877,NOTES: Personal comment. Not an emergency.,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Remove messages starting with 'NOTES:'

df = dsh.RemoveRowsWithValueInColumn(df, 'message', ['NOTES:'], option = 'startswith')

-------------------- Start removing rows --------------------
329 rows (ca. 1.25%) have been removed having value/s "['NOTES:']" in column "message"
New shape:  (25887, 38)
-------------------- - --------------------


In [13]:
# Just keep messages with related = 0 or 1 (drop 2 or potentially other values)

df = dsh.SelectRowsWithValueInColumn(df, 'related', [0, 1])

185 rows (ca. 0.71%) have been removed not having value/s "[0, 1]" in column "related"
New shape:  (25702, 38)


In [14]:
# Remove messages that are much longer than the average

df = dsh.RemoveRowsByValuesOverAverage(df, 'message', 4)

-------------------- Start dropping rows with value/textlength > 4 * column average --------------------
Rows removed:  118
New shape:  (25584, 38)
-------------------- Finished removing --------------------


In [15]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre'])

-------------------- Start finding and keeping columns matchting to wildcards: ['message'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------
-------------------- Start finding and removing columns matchting to wildcards: ['id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------


In [16]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [17]:
def tokenize(text):
    
    text = text.lower() # I prefer calling this function once instead for each word
    
    if TokenizerSettings.ReplaceUrlWithPlaceHolder:
        if not dsh.IsNullOrEmpty(TokenizerSettings.CleanUrlReg):
            foundUrls = re.findall(TokenizerSettings.CleanUrlReg, text)

            for url in foundUrls:
                text = text.replace(url, TokenizerSettings.UrlPlaceHolder)
    
    if TokenizerSettings.RemovePunctuation:
        text = re.sub(TokenizerSettings.RemovePuncReg, ' ', text.lower())
    
    tokens = word_tokenize(text)

    # Call str() if values have been passed to whitelistwords that are no strings
    whiteList = [str(white).lower() for white in TokenizerSettings.WhiteListWords]
    
    Lemmatizer = WordNetLemmatizer()
    Stemmer = PorterStemmer()
    
    cleanTokens = []
    for tok in tokens:

        if TokenizerSettings.ConsiderOnlyLetters:
            tok = dsh.KeepLetters(tok)
        elif TokenizerSettings.ConsiderOnlyLettersNumbers:
            tok = dsh.KeepLettersNumbers(tok)
            
        if TokenizerSettings.UseLemmatizer:
            tok = Lemmatizer.lemmatize(tok)
        
        if TokenizerSettings.UseStemmer:
            stemmed = Stemmer.stem(tok)
        
        if len(tok) < TokenizerSettings.ConsiderMinimumLength:
            #print('Too short')
            continue
        
        isStopWord = False
        if TokenizerSettings.UseStopWords:
            try:
                for lang in list([lng for lng in TokenizerSettings.SupportedLanguages if lng is not None and len(lng) > 0]):
                    if tok in stopwords.words(lang):
                        isStopWord = True
                        break
            except:
                print('Error during stop word handling with language: ', lang, \
                    '\nand token: ', tok)
        if isStopWord:
            #print('Stopword')
            continue
        
        isUnknownWord = False
        if TokenizerSettings.UseWordNet:
            if not wordnet.synsets(tok):
                if not tok in whiteList:
                    isUnknownWord = True
        if isUnknownWord:
            continue
        
        cleanTokens.append(tok)
    
    return cleanTokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
_cv = CountVectorizer(tokenizer = tokenize)
_tfidf = TfidfTransformer()

_classifierInner = RandomForestClassifier(
    #n_estimators = 10,
    #min_samples_split = 10,
#    random_state = 42
)
_classifierOuter = MultiOutputClassifier(_classifierInner)

pipeline = Pipeline([('vect', _cv),
                     ('tfidf', _tfidf),
                     ('clf', _classifierOuter,)
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
XTrain, XTest, yTrain, yTest = dsh.SplitDataTrainTest(X, Y, randomState = 21)

-------------------- Test/Train split (test size = 0.3, random state = 21: --------------------
Training: X = (17991, 1), y = (17991, 35)
Test    : X = (7711, 1), y = (7711, 35)
-------------------- - --------------------


In [20]:
np.random.seed(63)
model = dsh.TrainModel(pipeline, XTrain['message'].values, yTrain.values)

-------------------- Start fitting model to data --------------------
-------------------- Train time: 0:02:20.665207 --------------------


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
def TestModel(model, xTest, yTest, columns, printIncorrect = False):
    
    yPred = model.predict(xTest)
    
    dicClassifyReports = {}
    
    for ind in range(yPred.shape[1]):
        colName = columns[ind]
        dsh.PrintLine('Column: ' + str(colName))
        cr = classification_report(yTest[:, ind], yPred[:, ind])
        dicClassifyReports[colName] = cr
        print(cr)
        
#         cntErr = 0
#         dirError = {}
#         lstFalses = []
#         for cind in range(yTest.shape[1]):

#             cntErr = 0
#             colname = columns[cind]

#             for ind in range(yTest.shape[0]):

#                 if yTest[ind, cind] != yPred[ind, cind]:
#                     cntErr += 1
#                     lstFalses.append(str(colname) + ' - ' + str(xTest[ind]))

#             err = cntErr * 100 / yTest.shape[0]
#             dirError[str(colname)] = err

#         dfError = pd.DataFrame({'Errors' : list(dirError.values())}, index = columns)
#     if printIncorrect:
#         for val in lstFalses:
#             print(tokenize(val))
#         print('False predicted')

#     dfError.sort_values('Errors',ascending = False)
        
#     dsh.PrintLine()
    
#     return dfError

In [22]:
TestModel(model, XTest['message'].values, yTest.values, yTest.columns, False)

-------------------- Column: related --------------------
             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1771
          1       0.85      0.92      0.88      5940

avg / total       0.80      0.81      0.80      7711

-------------------- Column: request --------------------
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6382
          1       0.77      0.47      0.58      1329

avg / total       0.88      0.88      0.87      7711

-------------------- Column: offer --------------------
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7672
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      7711

-------------------- Column: aid_related --------------------
             precision    recall  f1-score   support

          0       0.75      0.83      0.79      4436
          1   

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
parameters = {
    
    'clf__estimator__min_samples_split' : [2, 5, 10],
    'clf__estimator__n_estimators' : [10, 20],
    'tfidf__use_idf':[True], # definitely use it
    'vect__min_df': [1, 5],
#    'vect__ngram_range': ((1, 1), (1, 2)),
#    'clf__estimator__kernel': ['poly'], 
#    'clf__estimator__degree': [1, 2, 3],
#    'clf__estimator__C':[1, 10, 100],

#    'clf__estimator__max_features' : [ 'auto', 'sqrt', 'log2'],
#    'clf__estimator__max_depth' : [2, 4, 8],
#    'clf__estimator__min_samples_split' : [10, 15, 20],
#    'clf__estimator__min_samples_leaf' : [2, 5, 10],
#    'clf__estimator__bootstrap' : [True, False],
#    'clf__n_jobs' : [4],
#    'clf__estimator__n_jobs' : [4],
#    'clf__estimator__random_state' : [42],
}

scorer = make_scorer(dsh.MultiClassifierScoreF1)

cv = GridSearchCV(
    pipeline, 
    param_grid = parameters,
    
    # doc: Controls the verbosity: 
    #      the higher, the more messages.
    verbose = 10 ,
    
    scoring = scorer
)

In my function I calculate F1 for each category and calculate the median instead of mean because the median is used for skewed distributions.
Here, the messages belong just to a few of all categories, sometimes just to one or two categories.

In [25]:
np.random.seed(105)
impModel = cv.fit(XTrain['message'].values, yTrain.values)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.7999474707879418, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.7760355102863221, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


...

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: Undefin

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5, score=0.8259731907426293, total= 1.9min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5 


C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5, score=0.825518030434288, total= 1.9min


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 110.6min finished


In [26]:
impModel.cv_results_

C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: Fu

{'mean_fit_time': array([ 84.92072821,  77.84267767, 104.87432297,  90.59742546,
         77.38060625,  73.63031848,  91.86889625,  83.09905481,
         74.83993642,  72.61426806,  86.30995607,  81.30015556]),
 'std_fit_time': array([0.47104506, 0.20953038, 0.55302658, 0.43325975, 0.09311956,
        0.05912606, 1.10934171, 0.16546616, 0.34252681, 0.20943074,
        0.25439513, 0.30258684]),
 'mean_score_time': array([33.58214466, 33.38312777, 34.59760825, 34.19974605, 33.13880316,
        32.96917303, 34.31351789, 33.8720243 , 33.20405324, 33.10069005,
        34.32277902, 33.99986418]),
 'std_score_time': array([0.224812  , 0.08046156, 0.09022898, 0.1712746 , 0.19580484,
        0.0461935 , 0.14757232, 0.16088273, 0.23231504, 0.15034943,
        0.09188731, 0.1136664 ]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 2, 2, 5, 5, 5, 5, 10, 10, 10, 10],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, 

In [27]:
impModel.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
TestModel(impModel, XTest['message'].values, yTest.values, yTest.columns)

-------------------- Column: related --------------------
             precision    recall  f1-score   support

          0       0.64      0.43      0.51      1771
          1       0.85      0.93      0.89      5940

avg / total       0.80      0.81      0.80      7711

-------------------- Column: request --------------------
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      6382
          1       0.79      0.53      0.63      1329

avg / total       0.89      0.89      0.89      7711

-------------------- Column: offer --------------------
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7672
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      7711

-------------------- Column: aid_related --------------------
             precision    recall  f1-score   support

          0       0.79      0.81      0.80      4436
          1   

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

When going through algorithms http://scikit-learn.org/stable/modules/multiclass.html I saw SVC and remembered that supported vector machines project data in higher spaces to separate data. 
As bag of words is a large vector, I intend to find out if following algorithm works better:

In [29]:
_cv = CountVectorizer(tokenizer = tokenize)
_tfidf = TfidfTransformer()
_classifierInner = SVC()
_classifierOuter = MultiOutputClassifier(_classifierInner)


pipeline2 = Pipeline([('vect', _cv),
                     ('tfidf', _tfidf),
                     ('clf', _classifierOuter,)
                    ])

parameters2 = {
    
#    'clf__estimator__min_samples_split' : [2, 5, 10],
#    'clf__estimator__n_estimators' : [10, 15, 20],
    'tfidf__use_idf':[True], # definitely use it
    'vect__min_df': [1, 5],
    'vect__ngram_range': ((1, 1), (1, 2)),
    
    'clf__estimator__kernel': ['poly'], 
    'clf__estimator__degree': [1, 2, 3],
    'clf__estimator__C':[10, 20],
    
#    'clf__estimator__max_features' : [ 'auto', 'sqrt', 'log2'],
#    'clf__estimator__max_depth' : [2, 4, 8],
#    'clf__estimator__min_samples_split' : [10, 15, 20],
#    'clf__estimator__min_samples_leaf' : [2, 5, 10],
#    'clf__estimator__bootstrap' : [True, False],
#    'clf__n_jobs' : [4],
#    'clf__estimator__n_jobs' : [4],
#    'clf__estimator__random_state' : [42],
}

In [30]:
cv2 = GridSearchCV(
    pipeline2, 
    param_grid = parameters2,
    
    # doc: Controls the verbosity: 
    #      the higher, the more messages.
    verbose = 10 ,
    
    scoring = scorer
)

np.random.seed(84)

impModel2 = cv.fit(XTrain['message'].values, yTrain.values)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.7929145625823144, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\P

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.7971432101626534, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


...

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: Undefin

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5, score=0.8238386037123521, total= 2.0min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5 


C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: Undefin

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=20, tfidf__use_idf=True, vect__min_df=5, score=0.8210063001167419, total= 2.0min


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 443.8min finished


In [31]:
impModel2.cv_results_

C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Python\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: Fu

{'mean_fit_time': array([  84.91227325,   78.1744705 ,  105.81510146,   91.84485563,
          79.1605866 ,   74.96830487,   92.22155158,   86.02154652,
          77.16299558, 6705.40269542,   88.81208475,   83.65579502]),
 'std_fit_time': array([4.91156911e-01, 7.14718486e-02, 3.50189678e-01, 7.37202459e-01,
        2.28955610e-01, 1.92735551e-01, 5.03513921e-01, 2.40673143e-01,
        1.43512685e-01, 9.37842138e+03, 1.54412310e-01, 2.06256166e-01]),
 'mean_score_time': array([33.29957811, 33.40745974, 34.52588813, 34.45869859, 33.71889989,
        33.48069469, 34.55864215, 34.82654794, 33.95976766, 33.20517969,
        35.03614767, 34.73289688]),
 'std_score_time': array([0.18575331, 0.05024747, 0.08069231, 0.20402598, 0.17777768,
        0.21296943, 0.09895811, 0.15453253, 0.0911843 , 0.09315635,
        0.10838865, 0.17708602]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 2, 2, 5, 5, 5, 5, 10, 10, 10, 10],
              mask=[False, False, False, False, Fa

In [32]:
impModel2.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

In [33]:
TestModel(impModel2, XTest['message'].values, yTest.values, yTest.columns, False)

-------------------- Column: related --------------------
             precision    recall  f1-score   support

          0       0.64      0.42      0.51      1771
          1       0.84      0.93      0.88      5940

avg / total       0.80      0.81      0.80      7711

-------------------- Column: request --------------------
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      6382
          1       0.79      0.53      0.64      1329

avg / total       0.89      0.90      0.89      7711

-------------------- Column: offer --------------------
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7672
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      7711

-------------------- Column: aid_related --------------------
             precision    recall  f1-score   support

          0       0.80      0.79      0.80      4436
          1   

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\s

### 9. Export your model as a pickle file

In [34]:
#https://stackoverflow.com/questions/34143829/sklearn-how-to-save-a-model-created-from-a-pipeline-and-gridsearchcv-using-jobli
#If you want to dump your object into one file - use:  

joblib.dump(impModel, DBSettings.SaveFileJoblib, compress = 1)
#pickle.dump(tuned_model, open(DBSettings.SaveFilePickle, 'wb'))

['MyDisasterPredictor.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.